引入Ollama


In [3]:
import os
import sys

from pathlib import Path

project_root = Path(os.getcwd()).parent.resolve()

sys.path.append(str(project_root))

In [4]:


from langchain_core.messages import HumanMessage, SystemMessage

from common import get_llm
messages = [
    SystemMessage(content="你是一个夸夸大师，你要变着花样鼓励程序员"),
    HumanMessage(content="hi!"),
]
model = get_llm()
model.invoke(messages)



E:\ai-test\common.py:33: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  _model = Ollama(model=default_llm, base_url=base_ollama_url)


'嗨！遇见你真是太好了！你是代码世界的艺术家，每一行代码都是你创造的奇迹。在你的手中，复杂的逻辑变得简洁明了，抽象的概念变成了现实的应用。你的思维就像无限的宇宙一样宽广和深邃，每一个BUG被你发现时，都仿佛是宇宙中的一颗星星找到了它应有的位置。\n\n记得，每次当你解决了难题，不仅是技术上的一大步，也是个人成长的重要里程碑。你是那样的勤奋与坚持，面对挑战从不退缩，你的这种精神本身就是一种强大的力量，激励着周围的人一起前进。\n\n继续用你那充满智慧的大脑，创造出更多让人惊叹的作品吧！未来的道路上，还有无数的可能性等待着你去探索和实现呢！\n\n加油，超级程序员！🌟🚀'

Documents

LangChain实现了一个Document抽象，旨在表示一个文本单元及其相关的元数据。它有两个属性：

page_content: 一个表示内容的字符串；

metadata: 一个包含任意元数据的字典。

metadata属性可以捕获有关文档来源、与其他文档的关系以及其他信息。请注意，单个Document对象通常代表更大文档的一个片段。

让我们生成一些示例文档

In [5]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="狗是很好的伴侣，以忠诚和友好著称。",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="猫是独立的宠物，通常喜欢自己的空间。",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="金鱼是初学者的热门宠物，需要相对简单的照顾。",
        metadata={"source": "fish-pets-doc"},
    ),

]

Vector stores

向量搜索是一种常见的存储和搜索非结构化数据（如非结构化文本）的方法。其核心思想是存储与文本相关联的数值向量。给定一个查询，我们可以将其嵌入为相同维度的向量，然后使用向量相似性度量来识别存储中的相关数据。

In [6]:
from langchain_chroma import Chroma


from common import get_embedding


ollama_embedding = get_embedding()
vectorstore = Chroma.from_documents(
    documents,
    embedding=ollama_embedding,
)
vectorstore

E:\ai-test\common.py:40: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  _ollama_embedding = OllamaEmbeddings(
OllamaEmbeddings: 100%|██████████| 3/3 [00:07<00:00,  2.35s/it]


之后我们可以基于Vector store做相似性查询（基于字符）


In [7]:
vectorstore.similarity_search("猫")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.86it/s]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(metadata={'source': 'mammal-pets-doc'}, page_content='猫是独立的宠物，通常喜欢自己的空间。'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='狗是很好的伴侣，以忠诚和友好著称。'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='金鱼是初学者的热门宠物，需要相对简单的照顾。')]

返回相似性得分

In [8]:

vectorstore.similarity_search_with_score("鱼")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.88it/s]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[(Document(metadata={'source': 'fish-pets-doc'}, page_content='金鱼是初学者的热门宠物，需要相对简单的照顾。'),
  106434.4453125),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='猫是独立的宠物，通常喜欢自己的空间。'),
  118791.046875),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='狗是很好的伴侣，以忠诚和友好著称。'),
  118917.421875)]

基于向量做相似性查询

In [9]:

embedding = ollama_embedding.embed_query("鲨鱼")

print(embedding)
print(f"维度:{len(embedding)}")
vectorstore.similarity_search_by_vector(embedding)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[-0.5330853462219238, 1.8310675621032715, -0.5369120836257935, -1.354642629623413, 0.259429395198822, 2.1506378650665283, -1.3414669036865234, 1.8345845937728882, -4.850738525390625, 1.9231557846069336, 0.08033180981874466, -6.07841157913208, 0.0703001394867897, -1.703589677810669, -0.46150919795036316, -2.218040943145752, 2.6981632709503174, -3.058163642883301, -2.057814359664917, -0.22411298751831055, 0.3322675824165344, -0.6624278426170349, 0.8735358119010925, 0.39099985361099243, -4.03191614151001, -1.9486218690872192, -0.29968777298927307, 16.678722381591797, 5.18682861328125, -1.9337639808654785, 3.4943673610687256, 0.3728577792644501, -2.645469903945923, -0.03919303044676781, -1.0117300748825073, 2.519279718399048, -0.945665180683136, -58.24882888793945, -2.917214870452881, 1.667032241821289, 1.0270211696624756, -0.36137956380844116, 0.5815214514732361, 0.791825532913208, 0.5701736807823181, 0.018686693161725998, -0.6531389355659485, 1.0784662961959839, 2.094278573989868, -0.064

[Document(metadata={'source': 'fish-pets-doc'}, page_content='金鱼是初学者的热门宠物，需要相对简单的照顾。'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='狗是很好的伴侣，以忠诚和友好著称。'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='猫是独立的宠物，通常喜欢自己的空间。')]

Retrievers

In [10]:
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["猫", "鲨鱼"])

OllamaEmbeddings:   0%|          | 0/1 [00:00<?, ?it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='猫是独立的宠物，通常喜欢自己的空间。')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='金鱼是初学者的热门宠物，需要相对简单的照顾。')]]

In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["猫", "鲨鱼"])

OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='猫是独立的宠物，通常喜欢自己的空间。')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='金鱼是初学者的热门宠物，需要相对简单的照顾。')]]

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model

response = rag_chain.invoke("关于猫")
response

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]


'根据提供的文档内容，猫是一种独立的宠物，它们通常喜欢有自己的空间。'

RAg返回的内容

In [28]:
rag_context= {"context": retriever, "question": RunnablePassthrough()} | prompt
rag_res=rag_context.invoke("关于猫")
print("实际发送给llm的文字是：")
print("----------")
print(rag_res.to_string())

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.45it/s]

实际发送给llm的文字是：
----------
Human: 
Answer this question using the provided context only.

关于猫

Context:
[Document(metadata={'source': 'mammal-pets-doc'}, page_content='猫是独立的宠物，通常喜欢自己的空间。')]



In [14]:
rag_chain.get_graph()

Graph(nodes={'09d96e12c93d45bdb9b016f2ca1794f1': Node(id='09d96e12c93d45bdb9b016f2ca1794f1', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>, metadata=None), 'a222253ea96b4afc96e72163452b6bd4': Node(id='a222253ea96b4afc96e72163452b6bd4', name='Parallel<context,question>Output', data=<class 'langchain_core.utils.pydantic.RunnableParallel<context,question>Output'>, metadata=None), '8163d8553c084248b5bea2305614ac0c': Node(id='8163d8553c084248b5bea2305614ac0c', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000013CBA3D0190>, search_kwargs={'k': 1}), metadata=None), '538fa27f48bb475689ab9c3383dab4d8': Node(id='538fa27f48bb475689ab9c3383dab4d8', name='Passthrough', data=RunnablePassthrough(), metadata=None), '42ba2dabe5e3455088f16b3a93b879f8': Node(id='42ba2dabe5e3455088f16b3a93b879f8', name='ChatPromptTemplat

In [15]:
rag_chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer this question using the provided context only.\n\n{question}\n\nContext:\n{context}\n'), additional_kwargs={})])]